### VERGE: vecotr-mode Regional Geospatial Embeddings
# Initialize Region Of Interest

In this project we will analyze a particular region of interest, defined by a
lon/lat bounding box. Here we save a dict with all ROI-specific parameters.

## Processing Setup

In [6]:
# # Google colab setup
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# project_home = '/content/drive/MyDrive/Projects/verge'
# os.chdir(project_home)

In [7]:
# Local processing setup
# project_home = '..'

In [23]:
# AWS / SageMaker
import sagemaker
project_home = 's3://odyssey-geospatial/verge'

## Code

In [24]:
import pyproj
import numpy as np
import json
import os

In [25]:
# ROI name and bounds

# All New England -- for the actual project
# roi_name = 'newengland'
# lat0, lon0 = 41.253746, -73.564321
# lat1, lon1 = 45.116468, -68.058533

# Small part of New England -- for development and testing
roi_name = 'ne-test'
lat0, lon0 = 41.253746, -73.564321
lat1, lon1 = 42.0, -72.3

# Size of tiles, in meters.
tile_size = 2000

# Resolution of encodings, in meters.
encoding_resolution = 100

# How much to shift consecutive tiles when tiling an AOI, in meters.
tile_shift = 1000

In [26]:
# The name of the general-purpose data directory.
data_home = '%s/data' % (project_home)
os.makedirs(data_home, exist_ok=True)

# The name of the ROI-specific data directory.
roi_home = '%s/data/%s' % (project_home, roi_name)
os.makedirs(roi_home, exist_ok=True)


In [27]:
# Get a good map projection for this ROI.
center_lat = (lat0 + lat1) / 2
center_lon = (lon0 + lon1) / 2

# Local transverse mercator projection.
proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon}
+k=1.0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs
"""
ltm_crs = pyproj.CRS.from_proj4(proj_def)

# Forward and inverse transforms with WGS84.
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

# Get the coords of the lower left corner.
x0, y0 = proj_forward(lon0, lat0)

# Get an offset.
x_offset = -np.floor(x0 / 1000) * 1000
y_offset = -np.floor(y0 / 1000) * 1000


In [28]:
# Re-define the local transverse mercator projection, with offsets.
proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon}
+k=1.0 +x_0={x_offset} +y_0={y_offset} +datum=WGS84 +units=m +no_defs
"""
ltm_crs = pyproj.CRS.from_proj4(proj_def)

# Forward and inverse transforms with WGS84.
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

# Get the coords of the lower left corner.
x0, y0 = proj_forward(lon0, lat0)
print(x0, y0)


17.41705232438835 752.3451999771642


In [29]:
print(proj_def)


+proj=tmerc +lat_0=41.626873 +lon_0=-72.93216050000001
+k=1.0 +x_0=53000.0 +y_0=42000.0 +datum=WGS84 +units=m +no_defs



In [30]:
# Put it all together.
roi = {
    'name': roi_name,
    'lon0': lon0,
    'lat0': lat0,
    'lon1': lon1,
    'lat1': lat1,
    'proj_def': proj_def,
    'tile_size': tile_size,
    'tile_shift': tile_shift,
    'encoding_resolution': encoding_resolution
}
print(json.dumps(roi, indent=4))

{
    "name": "ne-test",
    "lon0": -73.564321,
    "lat0": 41.253746,
    "lon1": -72.3,
    "lat1": 42.0,
    "proj_def": "\n+proj=tmerc +lat_0=41.626873 +lon_0=-72.93216050000001\n+k=1.0 +x_0=53000.0 +y_0=42000.0 +datum=WGS84 +units=m +no_defs\n",
    "tile_size": 2000,
    "tile_shift": 1000,
    "encoding_resolution": 100
}


In [31]:
import boto3

s3 = boto3.client("s3")  # Studio/Notebook already has creds/region
bucket = "odyssey-geospatial"
prefix = 'verge/data/%s' % roi_name
key = '%s/roi.json' % prefix

s3.put_object(
    Bucket=bucket,
    Key=key,
    Body=json.dumps(roi, ensure_ascii=False, separators=(",", ":")).encode("utf-8"),
    ContentType="application/json",
    # Optional hardening:
    # ServerSideEncryption="AES256",                    # or "aws:kms"
    # SSEKMSKeyId="arn:aws:kms:us-east-1:123:key/..."   # when using aws:kms
)

print(f"Wrote s3://{bucket}/{key}")


Wrote s3://odyssey-geospatial/verge/data/ne-test/roi.json


In [33]:
# Check by reading it back in.
obj = s3.get_object(Bucket=bucket, Key=key)
j = json.load(obj["Body"])
print(json.dumps(j, indent=4))


{
    "name": "ne-test",
    "lon0": -73.564321,
    "lat0": 41.253746,
    "lon1": -72.3,
    "lat1": 42.0,
    "proj_def": "\n+proj=tmerc +lat_0=41.626873 +lon_0=-72.93216050000001\n+k=1.0 +x_0=53000.0 +y_0=42000.0 +datum=WGS84 +units=m +no_defs\n",
    "tile_size": 2000,
    "tile_shift": 1000,
    "encoding_resolution": 100
}


In [ ]:
# fname = '%s/roi.json' % roi_home
# with open(fname, 'w') as dest:
#     json.dump(roi, dest, indent=4)
# print('saved %s' % fname)